<a href="https://colab.research.google.com/github/urvashi2004/PlantDoctor/blob/main/Colab/PlantDoctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import sys
import requests
from google.colab import drive, files

# Mount Google Drive
drive.mount('/content/drive')


def print_plant_doctor_title():
    title = """
    **************************************************
    *                                                *
    *          P L A N T  D O C T O R                *
    *                                                *
    *          __/\\__                                *
    *        /_/\\/\\/_\\                               *
    *          _/\\_                                  *
    *          /  \\          _                       *
    *         /    \\        (_)                      *
    *       /_/      \\_\\    | |                      *
    *                       | |                      *
    *                        V                       *
    *                                                *
    *       Your Plant Health Expert                 *
    *                                                *
    **************************************************
    """
    print(title)

def upload_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        return filename
    return None

def get_plant_identification(api_key, image_file):
    url = f"https://my-api.plantnet.org/v2/identify/all?include-related-images=true&no-reject=false&nb-results=5&lang=en&type=kt&api-key={api_key}"

    with open(image_file, 'rb') as img:
        files = {'images': img}

        response = requests.post(url, files=files)
        # print("API Response:", response.json())  # Debug print
        return response.json()

def format_results(results):
    output = ""
    if not results:
        return "No results found."

    for result in results:
        species = result.get("species", {})
        scientific_name = species.get("scientificName", "Unknown")
        common_names = ", ".join(species.get("commonNames", ["None"]))
        score = result.get("score", "Unknown") * 100  # Convert score to percentage
        images = result.get("images", [])

        output += f"Species: {scientific_name}\n"
        output += f"Common Names: {common_names}\n"
        output += f"Matching Score: {score:.2f}%\n"
        output += "Images:\n"

        for i, img in enumerate(images):  # Loop through all images
            output += f"Image {i + 1}: {img.get('url', {}).get('o', 'No URL')}\n"

        output += "\n"

    return output

def load_model():
    model_path = '/content/drive/My Drive/Colab Notebooks/PlantDoctor/PlantDiseaseModel.h5'
    if not os.path.isfile(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")
    model = tf.keras.models.load_model(model_path)
    return model

def predict_plant_health(image_path, model):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])

    if predicted_class == 0:
        return "Healthy Plant"
    else:
        return "Diseased Plant"

def main(api_key):
    print_plant_doctor_title()
    print("\nWelcome to PlantDoctor!")

    while True:
        print("\nMenu:")
        print("1. Identify Plant")
        print("2. Check Plant Health (Potato, Tomato, Pepper Bell)")
        print("3. Exit")

        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            image_file = upload_image()
            if image_file:
                print(f"Uploaded image file: {image_file}")
                response = get_plant_identification(api_key, image_file)
                results = response.get('results', [])
                # print("Raw Results:", results)  # Debug print
                formatted_results = format_results(results)
                print("Identification Results:\n")
                print(formatted_results)
            else:
                print("No image uploaded.")

        elif choice == '2':
            image_file = upload_image()
            if image_file:
                response = get_plant_identification(api_key, image_file)
                results = response.get('results', [])
                formatted_results = format_results(results)
                print(formatted_results)

                if results:
                    common_names = results[0].get('species', {}).get('commonNames', [])
                    common_names_str = ' '.join(common_names).lower()

                    if 'tomato' in common_names_str:
                        print("This plant is likely a Tomato")
                    elif 'potato' in common_names_str:
                        print("This plant is likely a Potato")
                    elif 'pepper bell' in common_names_str:
                        print("This plant is likely a Pepper Bell")
                    else:
                        print("This plant is not among the specified crops (Tomato, Potato, Pepper Bell).")

                    model = load_model()
                    health_status = predict_plant_health(image_file, model)
                    print(f"Prediction: {health_status}")
                else:
                    print("No results found.")

            else:
                print("No image uploaded.")

        elif choice == '3':
            print("Exiting PlantDoctor. Goodbye!")
            break

        else:
            print("Invalid choice, please select again.")

# Example usage
api_key = '2b10iBcUnPBl9JYeo7G1AKz9Su'  # Replace with your API key

Mounted at /content/drive


In [3]:
main(api_key)


    **************************************************
    *                                                *
    *          P L A N T  D O C T O R                *
    *                                                *
    *          __/\__                                *
    *        /_/\/\/_\                               *
    *          _/\_                                  *
    *          /  \          _                       *
    *         /    \        (_)                      *
    *       /_/      \_\    | |                      *
    *                       | |                      *
    *                        V                       *
    *                                                *
    *       Your Plant Health Expert                 *
    *                                                *
    **************************************************
    

Welcome to PlantDoctor!

Menu:
1. Identify Plant
2. Check Plant Health (Potato, Tomato, Pepper Bell)
3. Exit
Ente

Saving tomato-septoria-leaf-spot-grabowski.jpg to tomato-septoria-leaf-spot-grabowski (1).jpg
Uploaded image file: tomato-septoria-leaf-spot-grabowski (1).jpg
Identification Results:

Species: Solanum lycopersicum L.
Common Names: Garden tomato, Cherry Tomato, Tomato
Matching Score: 81.28%
Images:
Image 1: https://bs.plantnet.org/image/o/03198937bdccf2f73ed4e5af82d06c65c9b0e0d5
Image 2: https://bs.plantnet.org/image/o/e64f28aebb5cff10ebf4431f42b336f8a01ff992
Image 3: https://bs.plantnet.org/image/o/a0585e2263a532a2ba40d907d89d02a0711a6f59
Image 4: https://bs.plantnet.org/image/o/475a1200d2a8eb494a7caed85104fb8eb3c3a8ba
Image 5: https://bs.plantnet.org/image/o/1ba088855b0099f76d52352e7afd33a9030f32bc
Image 6: https://bs.plantnet.org/image/o/a78d3e767970b72f3526973a6d753873653a6119

Species: Valeriana tripteris L.
Common Names: Three-leaved Valerian, Valerian, Valériane à Feuilles Trifides (FR)
Matching Score: 0.66%
Images:
Image 1: https://bs.plantnet.org/image/o/32f2613d37e1a1107eb6f00

Saving tomato-septoria-leaf-spot-grabowski.jpg to tomato-septoria-leaf-spot-grabowski (2).jpg
Species: Solanum lycopersicum L.
Common Names: Garden tomato, Cherry Tomato, Tomato
Matching Score: 81.28%
Images:
Image 1: https://bs.plantnet.org/image/o/03198937bdccf2f73ed4e5af82d06c65c9b0e0d5
Image 2: https://bs.plantnet.org/image/o/e64f28aebb5cff10ebf4431f42b336f8a01ff992
Image 3: https://bs.plantnet.org/image/o/a0585e2263a532a2ba40d907d89d02a0711a6f59
Image 4: https://bs.plantnet.org/image/o/475a1200d2a8eb494a7caed85104fb8eb3c3a8ba
Image 5: https://bs.plantnet.org/image/o/1ba088855b0099f76d52352e7afd33a9030f32bc
Image 6: https://bs.plantnet.org/image/o/a78d3e767970b72f3526973a6d753873653a6119

Species: Valeriana tripteris L.
Common Names: Three-leaved Valerian, Valerian, Valériane à Feuilles Trifides (FR)
Matching Score: 0.66%
Images:
Image 1: https://bs.plantnet.org/image/o/32f2613d37e1a1107eb6f00103987be0fb6871f3

Species: Raphanus raphanistrum L.
Common Names: Wild radish, Radish, Se

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
Prediction: Diseased Plant

Menu:
1. Identify Plant
2. Check Plant Health (Potato, Tomato, Pepper Bell)
3. Exit
Enter your choice (1/2/3): 3
Exiting PlantDoctor. Goodbye!
